# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
load_csv = "output_data/cities.csv"
city_df = pd.read_csv(load_csv)
city_df.head()

FileNotFoundError: [Errno 2] File b'output_data/cities.csv' does not exist: b'output_data/cities.csv'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df['Humidity'].astype(int)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)

fig


NameError: name 'city_df' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
ideal_cond_city_df = city_df.copy()
ideal_cond_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 70]
ideal_cond_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 80]  
ideal_cond_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= 10]
ideal_cond_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] == 0]
ideal_cond_city_df = narrowed_city_df.dropna()
ideal_cond_city_df = narrowed_city_df.reset_index()
ideal_cond_city_df

NameError: name 'city_df' is not defined

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = ideal_cond_city_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df

NameError: name 'ideal_cond_city_df' is not defined

In [ ]:
hotels = []
x = 0
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

hotel_df['Hotel Name'] = hotels
hotel_df

In [ ]:
hotel_df = hotel_df.dropna()
hotel_df


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display Map
fig